## ***GMM Kumeleme Algoritmasi***

Bu ödevde, UC Irvine Machine Learning Repository'de bulunan [kalp hastalığı veri seti](http://archive.ics.uci.edu/ml/datasets/Heart+Disease)yle çalışacaksınız.
- n_components = 2 olarak ayarlayarak kalp hastalığı verilerine GMM uygulayın. Çözümünüz için ARI ve siluet skorlarına bakın ve önceki derslerde k-means ve hiyerarşik kümeleme çözümleriyle karşılaştırın. Hangi algoritma daha iyi performans gösteriyor?
- Scikit-learn ile GMM uygulanırken kullanılan parametrelerden biri de covariance_type'dir. Bu parametre, kullanılacak kovaryans parametrelerinin türünü belirler. Özellikle, belirtebileceğiniz dört tür vardır:
    - full: Bu varsayılandır. Her bileşenin kendi genel kovaryans matrisi vardır.
    - tied: Tüm bileşenler aynı genel kovaryans matrisini paylaşır.
    - diag: Her bir bileşenin kendi çapraz kovaryans matrisi vardır.
    - spherical: Her bir bileşenin kendine ait tek bir varyansı vardır.
- Bunları deneyin. Hangisi ARI ve siluet skorları açısından daha iyi performans gösteriyor?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from matplotlib.patches import Ellipse
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn import datasets, metrics

- Veri seti kümeleme algoritmasına geçmeden önce bazı işlemlere ihtiyaç duymaktadır.

#### Preprocessing

In [2]:
# birden fazla dosyayi okuma
import glob

path = r'./data' # use your path
all_files = glob.glob(path + "/processed*.data")

li = []

for filename in all_files:
    df = pd.read_csv(filename,header=0)
    # sutun isimlerini ayni yapiyoruz ki concat etmede sutun sayimiz artmasin
    # farkli isimdeki sutunlar olusmasin
    df.columns = [x for x in range(14)]
    li.append(df)

heart = pd.concat(li, axis=0, ignore_index=True)

In [3]:
heart

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,34.0,1.0,4.0,115,0,?,?,154,0,.2,1,?,?,1
1,35.0,1.0,4.0,?,0,?,0,130,1,?,?,?,7,3
2,36.0,1.0,4.0,110,0,?,0,125,1,1,2,?,6,1
3,38.0,0.0,4.0,105,0,?,0,166,0,2.8,1,?,?,2
4,38.0,0.0,4.0,110,0,0,0,156,0,0,2,?,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
911,45.0,1.0,1.0,110,264,0,0,132,0,1.2,2,0.0,7.0,1
912,68.0,1.0,4.0,144,193,1,0,141,0,3.4,2,2.0,7.0,2
913,57.0,1.0,4.0,130,131,0,0,115,1,1.2,2,1.0,7.0,3
914,57.0,0.0,2.0,130,236,0,2,174,0,0,2,1.0,3.0,1


- Burada, X sizin özellikleriniz olacak ve y içinde etiketleri tutuyoruz. Y, 1'e eşitse, o zaman buna karşılık gelen hastanın kalp hastalığı olduğunu ve y'nin 0'a eşit olması durumunda, hastanın kalp hastalığı olmadığı anlaşılacaktır.

In [4]:
# Veri setini kullanmadan önce bazı ön işlemler yapmanız gerekiyor:

# Define the features and the outcome
X = heart.iloc[:, :13]
y = heart.iloc[:, 13]

# Replace missing values (marked by ?) with a 0
X = X.replace(to_replace='?', value=0)

# Binarize y so that 1 means heart disease diagnosis and 0 means no diagnosis
y = np.where(y > 0, 0, 1)

- Kümeleme algoritmalalarının çoğu değişkenlerin ölçeklerine duyarlı olduklarından değişkenleri standartlaştırıyoruz

In [5]:
# kümeleme algoritmalalarının çoğu değişkenlerin ölçeklerine duyarlı olduklarından 
# değişkenleri standartlaştırıyoruz
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

---
#### ***Clustering***
- n_components=2 secilerek kumeleme olusturulur ve diger yontemlerle karsilastirilir. 

In [6]:
# epsilon ve min samples parametreleri ile kumeleme nesnesi olustur
gmm_cluster = GaussianMixture(n_components=2, random_state=123)

clusters = gmm_cluster.fit_predict(X_std)

In [13]:
print('GMM çözümü   :')
print('ARI skoru    : {:.6f}'
      .format(metrics.adjusted_rand_score(y, clusters)))
print("Siluet skoru : {:.6f}"
      .format(metrics.silhouette_score(X_std, clusters, metric='euclidean')))

GMM çözümü   :
ARI skoru    : 0.180288
Siluet skoru : 0.135165


- K-ortalama algoritmasi ile skorlar => Bknz:[k-means](https://github.com/ahmetpergamum/datascience/blob/master/11_clustering_algorithms/02_k-means_performance.ipynb)
    - ARI   : 0.23
    - Siluet: 0.15

- Hiyerarsik kumeleme algoritmasi ile skorlar=> Bknz:[hierarchical](https://github.com/ahmetpergamum/datascience/blob/master/11_clustering_algorithms/03_hierarchical_clustering.ipynb)

    - ARI   : 0.16 (en yuksek average linkage yontemi ile)
    - Siluet: 0.31 (en yuksek ward linkage yontemi ile)
    
- DBSCAN algoritmasi ile skorlar => Bknz:[dbscan](https://github.com/ahmetpergamum/datascience/blob/master/11_clustering_algorithms/04_dbscan_clustering.ipynb)
    - ARI   : 0.10 (test edilen parametrelerle elde edilen max)
    - Siluet: 0.27 (test edilen parametrelerle elde edilen max)

- GMM cozumunde ARI skoru K-ortalamadan dusuk Hiyerarsik ve DBSCAN den yuksek, Siluet skoru ise her uc yontemden de dusuk cikti.


---
#### ***Kovaryns Tipinin Kumelemeye Etkisi***

In [20]:
def cluster_n_score(cov_type):
    # kumeleme nesnesi olustur
    gmm_cluster = GaussianMixture(n_components=2, covariance_type=cov_type, random_state=123)
    clusters = gmm_cluster.fit_predict(X_std)
    # ARI ve siluet skorunu don
    return  metrics.adjusted_rand_score(y, clusters), metrics.silhouette_score(X_std, clusters, metric='euclidean')

In [21]:
cluster_n_score('full')

(0.1802882734028232, 0.13516545148979767)

In [24]:
cov_types = ['full','tied','diag', 'spherical' ]

for cov_type in  cov_types:
    ari, siluet = cluster_n_score(cov_type)
    print('Kovaryans tipi : {}'.format(cov_type))
    print('ARI skoru      : {:.4f}'.format(ari))
    print('Siluet skoru   : {:.4f}'.format(siluet))
    print('-'*20)
      

Kovaryans tipi : full
ARI skoru      : 0.1803
Siluet skoru   : 0.1352
--------------------
Kovaryans tipi : tied
ARI skoru      : 0.1478
Siluet skoru   : 0.1339
--------------------
Kovaryans tipi : diag
ARI skoru      : 0.1803
Siluet skoru   : 0.1352
--------------------
Kovaryans tipi : spherical
ARI skoru      : 0.2195
Siluet skoru   : 0.1529
--------------------


- En iyi skorlari spherical kovaryans tipinde elde ettik.

- Diger kumeleme algoritmalari ile karsilastirildiginda degerlerin K-ortalamaya cok yaklastigi(ARI skoru dusuk siluet skoru ayni), ARI skorunda Hiyerarsik ve DBSCAN den yuksek, Siluet skorunda ise Hiyerarsik ve DBSCAN den dusuk K-ortalamayla ayni sonuclar elde ettik.